In [67]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.impute import SimpleImputer

In [68]:
iowa_file_path = 'train.csv'

home_data = pd.read_csv(iowa_file_path)
# Create target object and call it y
# features_all = home_data.columns.tolist()
# features_all.remove('SalePrice')
# features = []
# for f in features_all:
#     if home_data[f].dtypes=="int64":
#         features.append(f)        
y = home_data.SalePrice
# Create X
#features = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
X = home_data.drop(['SalePrice'], axis=1) 
X = X.select_dtypes(exclude=['object'])



#X=X.drop(['LotFrontage', 'GarageYrBlt'], axis = 1)



# Split into validation and training data
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)


from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
train_X_with_imputed_values = my_imputer.fit_transform(train_X)
val_X_with_imputed_values = my_imputer.transform(val_X)


# Specify Model
iowa_model = DecisionTreeRegressor(random_state=1)




# Fit Model
iowa_model.fit(train_X_with_imputed_values, train_y)

# Make validation predictions and calculate mean absolute error
val_predictions = iowa_model.predict(val_X_with_imputed_values)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE when not specifying max_leaf_nodes: {:,.0f}".format(val_mae))

# Using best value for max_leaf_nodes
iowa_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)
iowa_model.fit(train_X_with_imputed_values, train_y)
val_predictions = iowa_model.predict(val_X_with_imputed_values)
val_mae = mean_absolute_error(val_predictions, val_y)
print("Validation MAE for best value of max_leaf_nodes: {:,.0f}".format(val_mae))

# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(max_leaf_nodes = 350,random_state=1)
rf_model.fit(train_X_with_imputed_values, train_y)
rf_val_predictions = rf_model.predict(val_X_with_imputed_values)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))



Validation MAE when not specifying max_leaf_nodes: 27,094
Validation MAE for best value of max_leaf_nodes: 24,931
Validation MAE for Random Forest Model: 18,877


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [69]:
def score_dataset(X_train, X_test, y_train, y_test):
    rf_model = RandomForestRegressor(max_leaf_nodes = 350, random_state=1)
    rf_model.fit(X_train, y_train)
    rf_val_predictions = rf_model.predict(X_test)
    rf_val_mae = mean_absolute_error(rf_val_predictions, y_test)
    return rf_val_mae

imputed_X_train_plus = train_X.copy()
imputed_X_test_plus = val_X.copy()

cols_with_missing = (col for col in train_X.columns 
                                 if train_X[col].isnull().any())
for col in cols_with_missing:
    imputed_X_train_plus[col + '_was_missing'] = imputed_X_train_plus[col].isnull()
    imputed_X_test_plus[col + '_was_missing'] = imputed_X_test_plus[col].isnull()

my_imputer = SimpleImputer()
imputed_X_train_plus = my_imputer.fit_transform(imputed_X_train_plus)
imputed_X_test_plus = my_imputer.transform(imputed_X_test_plus)

    
score_dataset(imputed_X_train_plus, imputed_X_test_plus, train_y, val_y)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


18745.87681724058

In [70]:
test_data_path = 'test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)

# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features

test_X = test_data.select_dtypes(exclude=['object'])

X_test_plus = test_X.copy()
X_train_plus = X.copy()

cols_with_missing = (col for col in X.columns 
                                 if X[col].isnull().any())

for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_test_plus[col + '_was_missing'] = X_test_plus[col].isnull()



#my_imputer = SimpleImputer()
imputed_X_train_plus = my_imputer.fit_transform(X_train_plus)
imputed_X_test_plus = my_imputer.transform(X_test_plus)


rf_model = RandomForestRegressor(max_leaf_nodes = 350, random_state=1)
rf_model.fit(imputed_X_train_plus, y)
test_preds = rf_model.predict(imputed_X_test_plus)
#rf_val_mae = mean_absolute_error(rf_val_predictions, y_test)



# The lines below shows you how to save your data in the format needed to score it in the competition
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})

output.to_csv('submission.csv', index=False)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
